# 1.Import library

In [38]:
import pandas as pd
import numpy as np
import seaborn as sns
import scipy

# 2.데이터 병합 및 분할

In [33]:
business = pd.read_csv('rental_member_data_buisness.csv')
personal = pd.read_csv('rental_member_data_personal.csv')
sales = pd.read_csv('rental_sales_data.csv')

business_personal = pd.concat([business,personal])
sales_t = pd.merge(business_personal , sales , on = ['고객ID'],how = 'right')
sales_t.head()
# sales_t.info()

,고객ID,고객유형,생년,생월,거주지,시,구군,남여구분,판매유형,계약유형,...,계약기간,납부유형,제품군,월랜탈비용,계약상태,연체건수,연체여부,신용등급,결제은행,해약여부
0,25702131,사업자,NaN,NaN,경기도,경기도,화성시,여자,멤버십,멤버십S,...,12,CMS,DES-1,66900,계약확정,0,없음,NaN,기업은행,정상
1,25742841,사업자,1964.0,8.0,경기도,경기도,가평군,여자,렌탈,프로모션,...,60,CMS,DES-1,142200,계약확정,0,없음,NaN,농협회원조합,정상
2,25742683,사업자,1973.0,12.0,서울특별시,서울특별시,강남구,여자,렌탈,프로모션,...,60,CMS,DES-3A,102900,계약확정,0,없음,NaN,신한은행,정상
3,25742683,사업자,1973.0,12.0,서울특별시,서울특별시,강남구,여자,렌탈,프로모션,...,60,CMS,DES-3A,102900,계약확정,0,없음,NaN,신한은행,정상
4,25851817,사업자,NaN,NaN,서울특별시,서울특별시,강남구,여자,멤버십,멤버십S,...,12,CMS,DES-1,96600,계약확정,0,없음,NaN,국민은행,정상


In [34]:
sales_t['고객유형'] = sales_t['고객유형'].str.split(' ').str[1]

In [35]:
a = pd.crosstab(sales_t['고객유형'], sales_t['해약여부'], margins=True)
a

해약여부,정상,해약,All
고객유형,,,
개인,45344,903,46247
사업자,5031,4,5035
All,50375,907,51282


In [37]:
c1 = [a.loc['개인'][0],a.loc['개인'][1]]
c2 = [a.loc['사업자'][0],a.loc['사업자'][1]]
_, p_value, _, _ = scipy.stats.chi2_contingency([c1, c2])

print('카이제곱 검정 결과')
print('pvalue:{:.3f}'.format(p_value))

카이제곱 검정 결과
pvalue:0.000


In [15]:
df_p = sales_t[sales_t['고객유형'] == '개인']
df_b = sales_t[sales_t['고객유형'] == '사업자']

# 3.B2C

# 파이썬에서의 전처리 

In [49]:
# df_p.head()
# df_p.info()  # 46269
# df_p.describe() # 생년 이상치 발견 

In [85]:
df_p.isnull().sum()

# 결측비율
# 생년 4041/46269 = 8.73%
# 신용등급 3780/46269 = 8.16% 
# 결제은행 2084/46269 = 4.5%

고객ID        0
고객유형        0
생년       4041
생월       4041
거주지         1
시           0
구군          0
남여구분        0
판매유형        4
계약유형        1
유통경로        0
계약일         0
계약기간        0
납부유형        0
제품군         1
월랜탈비용       0
계약상태        0
연체건수        0
연체여부        2
신용등급     3780
결제은행     2084
해약여부       22
dtype: int64

In [86]:
df_new_p = df_p
# df_new_p.head()

In [87]:
# 생월 삭제
df_new_p = df_new_p.drop(['생월'],axis = 1,inplace = False)  

In [88]:
# 거주지 결측치 확인 및 대체
df_new_p.loc[df_new_p['거주지'].isnull()==True,:]

,고객ID,고객유형,생년,거주지,시,구군,남여구분,판매유형,계약유형,유통경로,...,계약기간,납부유형,제품군,월랜탈비용,계약상태,연체건수,연체여부,신용등급,결제은행,해약여부
5316,66758696,개인,1988.0,NaN,광주광역시,북구,여자,렌탈,일반,렌탈총판,...,60,CMS,DES-1,105900,계약확정,0,없음,4.0,국민은행,정상


In [89]:
df_new_p['거주지'].fillna('전라도', inplace = True)

In [90]:
# 이상치 확인 및 대체
df_new_p.loc[df_new_p['연체건수'] == -1,'연체건수']  = np.nan

df_new_p['연체건수'].fillna(0, inplace = True)

In [91]:
df_new_p.loc[df_new_p['계약기간'] == 600,'계약기간']  = np.nan  #1개
df_new_p.loc[df_new_p['계약기간'] == 600000,'계약기간']  = np.nan  #1개

df_new_p['계약기간'].fillna(60, inplace = True)

In [92]:
df_new_p.loc[df_new_p['월랜탈비용'] == 999999999,'월랜탈비용']  = np.nan  #1개

df_new_p['월랜탈비용'].fillna(96900, inplace = True)  # median으로 대체

In [93]:
df_new_p.loc[df_new_p['생년'] == 1999978,'생년']  = np.nan  
df_new_p['생년'].fillna(1978, inplace = True)  

In [94]:
df_new_p.loc[df_new_p['생년'] == 199982,'생년']  = np.nan  
df_new_p['생년'].fillna(1982, inplace = True)  

In [95]:
df_new_p.loc[df_new_p['생년'] == 195,'생년']  = np.nan  
df_new_p['생년'].fillna(1950, inplace = True)  

In [96]:
df_new_p.loc[df_new_p['생년'] == 197,'생년']  = np.nan  
df_new_p['생년'].fillna(1970, inplace = True)  

In [100]:
df_new_p.isnull().sum()

고객ID        0
고객유형        0
생년          0
거주지         0
시           0
구군          0
남여구분        0
판매유형        4
계약유형        1
유통경로        0
계약일         0
계약기간        0
납부유형        0
제품군         1
월랜탈비용       0
계약상태        0
연체건수        0
연체여부        2
신용등급     3780
결제은행     2084
해약여부       22
dtype: int64

In [101]:
# df_new_p.to_csv('personal.csv')  # 지금까지의 전처리 과정을 csv파일로 외부로 저장

# R 에서의 전처리(MICE) 

In [102]:
# omit

# 파이썬에서의 추가 전처리(파생변수)

In [39]:
df_new_p = pd.read_csv('new_personal.csv')
df_new_p.head()

,Unnamed: 0,X,고객ID,고객유형,생년,거주지,시,구군,남여구분,판매유형,...,계약기간,납부유형,제품군,월랜탈비용,계약상태,연체건수,연체여부,신용등급,결제은행,해약여부
0,1,5036,66758234,개인,1979,경기도,경기도,평택시,여자,렌탈,...,60,CMS,DES-1,96900,계약확정,0,없음,9,새마을금고,정상
1,2,5037,66755948,개인,1982,경기도,경기도,평택시,남자,렌탈,...,60,카드이체,DES-1,102900,계약확정,0,없음,2,현대카드,정상
2,3,5038,66755948,개인,1982,경기도,경기도,평택시,남자,렌탈,...,60,카드이체,DES-3A,90900,계약확정,0,없음,2,현대카드,정상
3,4,5039,66756657,개인,1973,경기도,경기도,평택시,여자,렌탈,...,60,CMS,DES-1,96900,계약확정,0,없음,8,우리은행,정상
4,5,5040,66423450,개인,1982,경기도,경기도,포천시,남자,멤버십,...,12,CMS,DES-1,66900,계약확정,0,없음,5,농협회원조합,정상


In [40]:
df_new_p = df_new_p.drop(['Unnamed: 0','X'],axis = 1,inplace = False)  
df_new_p.head()
# df_new_p.info()

,고객ID,고객유형,생년,거주지,시,구군,남여구분,판매유형,계약유형,유통경로,...,계약기간,납부유형,제품군,월랜탈비용,계약상태,연체건수,연체여부,신용등급,결제은행,해약여부
0,66758234,개인,1979,경기도,경기도,평택시,여자,렌탈,일반,서비스 방문,...,60,CMS,DES-1,96900,계약확정,0,없음,9,새마을금고,정상
1,66755948,개인,1982,경기도,경기도,평택시,남자,렌탈,교체렌탈,서비스 방문,...,60,카드이체,DES-1,102900,계약확정,0,없음,2,현대카드,정상
2,66755948,개인,1982,경기도,경기도,평택시,남자,렌탈,프로모션,서비스 방문,...,60,카드이체,DES-3A,90900,계약확정,0,없음,2,현대카드,정상
3,66756657,개인,1973,경기도,경기도,평택시,여자,렌탈,일반,홈쇼핑/방송,...,60,CMS,DES-1,96900,계약확정,0,없음,8,우리은행,정상
4,66423450,개인,1982,경기도,경기도,포천시,남자,멤버십,멤버십S,렌탈재계약,...,12,CMS,DES-1,66900,계약확정,0,없음,5,농협회원조합,정상


In [41]:
df_new_p.isnull().sum()

고객ID     0
고객유형     0
생년       0
거주지      0
시        0
구군       0
남여구분     0
판매유형     0
계약유형     0
유통경로     0
계약일      0
계약기간     0
납부유형     0
제품군      0
월랜탈비용    0
계약상태     0
연체건수     0
연체여부     0
신용등급     0
결제은행     0
해약여부     0
dtype: int64

In [42]:
# 연령대변수 추가 

def age_cal(data):
    age1 = 2015-data
    age2 = age1%10
    
    if age2 == 0:
        return age1
    else:
        return age1-age2
        

df_new_p['연령대']= df_new_p['생년'].apply(age_cal) 
df_new_p.head()

,고객ID,고객유형,생년,거주지,시,구군,남여구분,판매유형,계약유형,유통경로,...,납부유형,제품군,월랜탈비용,계약상태,연체건수,연체여부,신용등급,결제은행,해약여부,연령대
0,66758234,개인,1979,경기도,경기도,평택시,여자,렌탈,일반,서비스 방문,...,CMS,DES-1,96900,계약확정,0,없음,9,새마을금고,정상,30
1,66755948,개인,1982,경기도,경기도,평택시,남자,렌탈,교체렌탈,서비스 방문,...,카드이체,DES-1,102900,계약확정,0,없음,2,현대카드,정상,30
2,66755948,개인,1982,경기도,경기도,평택시,남자,렌탈,프로모션,서비스 방문,...,카드이체,DES-3A,90900,계약확정,0,없음,2,현대카드,정상,30
3,66756657,개인,1973,경기도,경기도,평택시,여자,렌탈,일반,홈쇼핑/방송,...,CMS,DES-1,96900,계약확정,0,없음,8,우리은행,정상,40
4,66423450,개인,1982,경기도,경기도,포천시,남자,멤버십,멤버십S,렌탈재계약,...,CMS,DES-1,66900,계약확정,0,없음,5,농협회원조합,정상,30


In [43]:
df_new_p['연령대'].value_counts()

30    17412
40    14408
50     8011
60     2677
20     2665
70      905
80      184
90        6
10        1
Name: 연령대, dtype: int64

In [44]:
# 계정수변수 추가

In [45]:
a = df_new_p['고객ID'].value_counts()  # 46000-> 40000
# a.values
# a.index

In [46]:
a = df_new_p['고객ID'].value_counts()  # 46000-> 40000

df1 = pd.DataFrame(a.index, columns=['고객ID'])
df2 = pd.DataFrame(a.values, columns=['계정수'])
df = pd.merge(df1, df2, left_index=True, right_index=True)

df_new_p = pd.merge(df_new_p ,df, on = ['고객ID'],how = 'right')
df_new_p.head()

,고객ID,고객유형,생년,거주지,시,구군,남여구분,판매유형,계약유형,유통경로,...,제품군,월랜탈비용,계약상태,연체건수,연체여부,신용등급,결제은행,해약여부,연령대,계정수
0,66758234,개인,1979,경기도,경기도,평택시,여자,렌탈,일반,서비스 방문,...,DES-1,96900,계약확정,0,없음,9,새마을금고,정상,30,1
1,66755948,개인,1982,경기도,경기도,평택시,남자,렌탈,교체렌탈,서비스 방문,...,DES-1,102900,계약확정,0,없음,2,현대카드,정상,30,2
2,66755948,개인,1982,경기도,경기도,평택시,남자,렌탈,프로모션,서비스 방문,...,DES-3A,90900,계약확정,0,없음,2,현대카드,정상,30,2
3,66756657,개인,1973,경기도,경기도,평택시,여자,렌탈,일반,홈쇼핑/방송,...,DES-1,96900,계약확정,0,없음,8,우리은행,정상,40,1
4,66423450,개인,1982,경기도,경기도,포천시,남자,멤버십,멤버십S,렌탈재계약,...,DES-1,66900,계약확정,0,없음,5,농협회원조합,정상,30,1


In [47]:
df_new_p.columns

Index(['고객ID', '고객유형', '생년', '거주지', '시', '구군', '남여구분', '판매유형', '계약유형', '유통경로',
       '계약일', '계약기간', '납부유형', '제품군', '월랜탈비용', '계약상태', '연체건수', '연체여부', '신용등급',
       '결제은행', '해약여부', '연령대', '계정수'],
      dtype='object')

In [48]:
df_new_p['유통경로'].value_counts()

서비스 방문     13744
홈쇼핑/방송     12681
렌탈총판        4659
R관리방판       4163
하이마트        3142
영업방판        2332
렌탈재계약       1950
전자랜드        1127
일반          1008
R빌리미         458
이마트          248
홈플러스         212
R인터넷         189
홈쇼핑/인터넷      171
R렌탈운영        107
R농협인터넷몰       76
R법인그룹          2
Name: 유통경로, dtype: int64

In [49]:
df_new_p.loc[df_new_p['유통경로'] == '홈쇼핑/방송' ,'유통경로'] = np.nan
df_new_p['유통경로'].fillna('홈쇼핑', inplace = True)  

In [50]:
df_new_p.loc[df_new_p['유통경로'] == 'R인터넷' ,'유통경로'] = np.nan
df_new_p['유통경로'].fillna('온라인', inplace = True)  

In [51]:
df_new_p.loc[df_new_p['유통경로'] == 'R농협인터넷몰' ,'유통경로'] = np.nan
df_new_p['유통경로'].fillna('온라인', inplace = True)  

In [52]:
df_new_p.loc[df_new_p['유통경로'] == '홈쇼핑/인터넷' ,'유통경로'] = np.nan
df_new_p['유통경로'].fillna('온라인', inplace = True)  

In [53]:
df_new_p.loc[df_new_p['유통경로'] == '이마트' ,'유통경로'] = np.nan
df_new_p['유통경로'].fillna('기기구매', inplace = True)  

In [54]:
df_new_p.loc[df_new_p['유통경로'] == '홈플러스' ,'유통경로'] = np.nan
df_new_p['유통경로'].fillna('기기구매', inplace = True)  

In [55]:
df_new_p.loc[df_new_p['유통경로'] == '일반' ,'유통경로'] = np.nan
df_new_p['유통경로'].fillna('기기구매', inplace = True)  

In [56]:
df_new_p.loc[df_new_p['유통경로'] == '서비스 방문' ,'유통경로'] = np.nan
df_new_p['유통경로'].fillna('방문판매', inplace = True)  

In [57]:
df_new_p.loc[df_new_p['유통경로'] == 'R관리방판' ,'유통경로'] = np.nan
df_new_p['유통경로'].fillna('방문판매', inplace = True)  

In [58]:
df_new_p.loc[df_new_p['유통경로'] == '영업방판' ,'유통경로'] = np.nan
df_new_p['유통경로'].fillna('방문판매', inplace = True)  

In [59]:
df_new_p.loc[df_new_p['유통경로'] == 'R법인그룹' ,'유통경로'] = np.nan
df_new_p['유통경로'].fillna('기타', inplace = True)  

In [60]:
df_new_p.loc[df_new_p['유통경로'] == 'R빌리미' ,'유통경로'] = np.nan
df_new_p['유통경로'].fillna('기타', inplace = True)    

In [61]:
df_new_p.loc[df_new_p['유통경로'] == 'R렌탈운영' ,'유통경로'] = np.nan
df_new_p['유통경로'].fillna('기타', inplace = True)  

In [62]:
df_new_p['유통경로'].value_counts()

방문판매     20239
홈쇼핑      12681
렌탈총판      4659
하이마트      3142
렌탈재계약     1950
기기구매      1468
전자랜드      1127
기타         567
온라인        436
Name: 유통경로, dtype: int64

In [64]:
a = pd.read_csv('시구군 지국수.csv')

In [78]:
a['시구군'].value_counts

<bound method IndexOpsMixin.value_counts of 0        강원도 강릉시
1        강원도 동해시
2        강원도 삼척시
3        강원도 속초시
4        강원도 영월군
5        강원도 원주시
6        강원도 인제군
7        강원도 춘천시
8        강원도 태백시
9        강원도 홍천군
10        경기도 경기
11       경기도 고양시
12       경기도 광명시
13       경기도 광주시
14       경기도 구리시
15       경기도 군포시
16       경기도 김포시
17      경기도 남양주시
18      경기도 동두천시
19       경기도 부천시
20       경기도 성남시
21       경기도 수원시
22       경기도 시흥시
23       경기도 안산시
24       경기도 안성시
25       경기도 안양시
26       경기도 양주시
27       경기도 양평군
28       경기도 여주군
29       경기도 오산시
         ...    
121     인천광역시 서구
122    인천광역시 연수구
123     인천광역시 중구
124     전라남도 광양시
125     전라남도 목포시
126     전라남도 순천시
127     전라남도 여수시
128     전라북도 군산시
129     전라북도 익산시
130     전라북도 전주시
131     전라북도 정읍시
132     제주도 서귀포시
133      제주도 제주시
134       충남 연기군
135     충청남도 계룡시
136     충청남도 공주시
137     충청남도 논산시
138     충청남도 당진군
139     충청남도 보령시
140     충청남도 부여군
141     충청남도 서산시
142     충청남도 아산시
143     충청남도 예산군
144     충청남도 천안시
145     충청남도 홍성군
146 

In [77]:
df_new_p['시구군'] = df_new_p['시'] + df_new_p['구군']
df_new_p['시구군'].value_counts()

경기도 남양주시       814
경기도 화성시        707
인천광역시 부평구      687
경상도 김해시        628
인천광역시 남동구      612
서울특별시 강서구      569
인천광역시 서구       561
서울특별시 송파구      560
경기도 파주시        515
서울특별시 노원구      508
경기도 성남시 분당구    501
경기도 평택시        494
서울특별시 강남구      481
경기도 고양시 덕양구    466
대구광역시 달서구      454
경기도 의정부시       449
경기도 김포시        444
경기도 광주시        439
부산광역시 해운대구     438
전라도 전주시 완산구    435
인천광역시 남구       434
경기도 시흥시        433
경기도 부천시 원미구    431
대전광역시 서구       426
서울특별시 은평구      422
경기도 수원시 권선구    420
광주광역시 광산구      418
경기도 용인시 기흥구    414
경기도 안산시 상록구    404
서울특별시 양천구      380
              ... 
강원도 평창군         16
충청도 단양군         16
강원도 횡성군         16
전라도 진도군         16
경상도 하동군         16
경상도 의성군         15
경상도 합천군         15
전라도 장수군         15
경상도 성주군         15
경상도 봉화군         15
전라도 곡성군         14
강원도 고성군         14
전라도 진안군         14
전라도 함평군         13
충청도 금산군         12
전라도 보성군         11
경상도 군위군         11
강원도 양구군         11
경상도 의령군         11
전라도 장성군         10
전라도 강진군          9
경상도 청송군     

In [31]:
df_new_p.to_csv('final_personal.csv')  # 최종 전처리 과정을 csv파일로 외부로 저장

# 4.B2B

# 파이썬에서의 전처리 

In [128]:
df_b.head()
# df_b.info()  # 5036

,고객ID,고객유형,생년,생월,거주지,시,구군,남여구분,판매유형,계약유형,...,계약기간,납부유형,제품군,월랜탈비용,계약상태,연체건수,연체여부,신용등급,결제은행,해약여부
0,25702131,사업자,NaN,NaN,경기도,경기도,화성시,여자,멤버십,멤버십S,...,12,CMS,DES-1,66900,계약확정,0,없음,NaN,기업은행,정상
1,25742841,사업자,1964.0,8.0,경기도,경기도,가평군,여자,렌탈,프로모션,...,60,CMS,DES-1,142200,계약확정,0,없음,NaN,농협회원조합,정상
2,25742683,사업자,1973.0,12.0,서울특별시,서울특별시,강남구,여자,렌탈,프로모션,...,60,CMS,DES-3A,102900,계약확정,0,없음,NaN,신한은행,정상
3,25742683,사업자,1973.0,12.0,서울특별시,서울특별시,강남구,여자,렌탈,프로모션,...,60,CMS,DES-3A,102900,계약확정,0,없음,NaN,신한은행,정상
4,25851817,사업자,NaN,NaN,서울특별시,서울특별시,강남구,여자,멤버십,멤버십S,...,12,CMS,DES-1,96600,계약확정,0,없음,NaN,국민은행,정상


In [129]:
df_b.isnull().sum()
# 결측비율
# 결제은행 676/5036 = 13%

고객ID        0
고객유형        0
생년       2928
생월       2928
거주지         0
시           1
구군          0
남여구분        0
판매유형        0
계약유형        0
유통경로        0
계약일         0
계약기간        0
납부유형        0
제품군         0
월랜탈비용       0
계약상태        0
연체건수        0
연체여부        0
신용등급     5003
결제은행      676
해약여부        1
dtype: int64

In [130]:
df_new_b = df_b
# df_new_p.head()

In [131]:
# 생월 생년 신용등급 삭제
df_new_b = df_new_b.drop(['생월','생년','신용등급'],axis = 1,inplace = False)  

In [132]:
df_new_b.loc[df_new_b['시'].isnull()==True,:]

,고객ID,고객유형,거주지,시,구군,남여구분,판매유형,계약유형,유통경로,계약일,계약기간,납부유형,제품군,월랜탈비용,계약상태,연체건수,연체여부,결제은행,해약여부
60,25742697,사업자,경기도,NaN,광주시,여자,렌탈,일반,서비스 방문,10-21-2014,60,CMS,DES-1,96900,계약확정,0,없음,국민은행,정상


In [133]:
df_new_b['시'].fillna('광주시', inplace = True)

In [136]:
df_new_b.isnull().sum()

고객ID       0
고객유형       0
거주지        0
시          0
구군         0
남여구분       0
판매유형       0
계약유형       0
유통경로       0
계약일        0
계약기간       0
납부유형       0
제품군        0
월랜탈비용      0
계약상태       0
연체건수       0
연체여부       0
결제은행     676
해약여부       1
dtype: int64

In [137]:
# df_new_b.to_csv('business.csv')  # 지금까지의 전처리 과정을 csv파일로 외부로 저장

# R 에서의 전처리(MICE) 

In [ ]:
# omit

# 파이썬에서의 추가 전처리(파생변수)

In [138]:
df_new_b = pd.read_csv('new_business.csv')
df_new_b.head()

,Unnamed: 0,X,고객ID,고객유형,거주지,시,구군,남여구분,판매유형,계약유형,...,계약일,계약기간,납부유형,제품군,월랜탈비용,계약상태,연체건수,연체여부,결제은행,해약여부
0,1,0,25702131,사업자,경기도,경기도,화성시,여자,멤버십,멤버십S,...,10-20-2014,12,CMS,DES-1,66900,계약확정,0,없음,기업은행,정상
1,2,1,25742841,사업자,경기도,경기도,가평군,여자,렌탈,프로모션,...,10-21-2014,60,CMS,DES-1,142200,계약확정,0,없음,농협회원조합,정상
2,3,2,25742683,사업자,서울특별시,서울특별시,강남구,여자,렌탈,프로모션,...,10-21-2014,60,CMS,DES-3A,102900,계약확정,0,없음,신한은행,정상
3,4,3,25742683,사업자,서울특별시,서울특별시,강남구,여자,렌탈,프로모션,...,11-19-2014,60,CMS,DES-3A,102900,계약확정,0,없음,신한은행,정상
4,5,4,25851817,사업자,서울특별시,서울특별시,강남구,여자,멤버십,멤버십S,...,10-21-2014,12,CMS,DES-1,96600,계약확정,0,없음,국민은행,정상


In [139]:
df_new_b = df_new_b.drop(['Unnamed: 0','X'],axis = 1,inplace = False)  
df_new_b.head()
# df_new_b.info()

,고객ID,고객유형,거주지,시,구군,남여구분,판매유형,계약유형,유통경로,계약일,계약기간,납부유형,제품군,월랜탈비용,계약상태,연체건수,연체여부,결제은행,해약여부
0,25702131,사업자,경기도,경기도,화성시,여자,멤버십,멤버십S,렌탈재계약,10-20-2014,12,CMS,DES-1,66900,계약확정,0,없음,기업은행,정상
1,25742841,사업자,경기도,경기도,가평군,여자,렌탈,프로모션,서비스 방문,10-21-2014,60,CMS,DES-1,142200,계약확정,0,없음,농협회원조합,정상
2,25742683,사업자,서울특별시,서울특별시,강남구,여자,렌탈,프로모션,일반,10-21-2014,60,CMS,DES-3A,102900,계약확정,0,없음,신한은행,정상
3,25742683,사업자,서울특별시,서울특별시,강남구,여자,렌탈,프로모션,일반,11-19-2014,60,CMS,DES-3A,102900,계약확정,0,없음,신한은행,정상
4,25851817,사업자,서울특별시,서울특별시,강남구,여자,멤버십,멤버십S,렌탈재계약,10-21-2014,12,CMS,DES-1,96600,계약확정,0,없음,국민은행,정상


In [141]:
df_new_b.isnull().sum()

고객ID     0
고객유형     0
거주지      0
시        0
구군       0
남여구분     0
판매유형     0
계약유형     0
유통경로     0
계약일      0
계약기간     0
납부유형     0
제품군      0
월랜탈비용    0
계약상태     0
연체건수     0
연체여부     0
결제은행     0
해약여부     0
dtype: int64

In [ ]:
# 계정수변수 추가

In [144]:
a = df_new_b['고객ID'].value_counts()  # 46000-> 40000
# a.values
# a.index

Int64Index([25777820, 25869517, 25733179, 25722644, 25863590, 25742586,
            25721902, 25731964, 25731616, 25742699,
            ...
            25800766, 25852969, 25734406, 25855014, 25848867, 25740322,
            25768988, 25855002, 25738265, 25733125],
           dtype='int64', length=3590)

In [145]:
df1 = pd.DataFrame(a.index, columns=['고객ID'])
df2 = pd.DataFrame(a.values, columns=['계정수'])
df = pd.merge(df1, df2, left_index=True, right_index=True)

df_new_b = pd.merge(df_new_b ,df, on = ['고객ID'],how = 'right')
df_new_b.head()

,고객ID,고객유형,거주지,시,구군,남여구분,판매유형,계약유형,유통경로,계약일,계약기간,납부유형,제품군,월랜탈비용,계약상태,연체건수,연체여부,결제은행,해약여부,계정수
0,25702131,사업자,경기도,경기도,화성시,여자,멤버십,멤버십S,렌탈재계약,10-20-2014,12,CMS,DES-1,66900,계약확정,0,없음,기업은행,정상,1
1,25742841,사업자,경기도,경기도,가평군,여자,렌탈,프로모션,서비스 방문,10-21-2014,60,CMS,DES-1,142200,계약확정,0,없음,농협회원조합,정상,1
2,25742683,사업자,서울특별시,서울특별시,강남구,여자,렌탈,프로모션,일반,10-21-2014,60,CMS,DES-3A,102900,계약확정,0,없음,신한은행,정상,2
3,25742683,사업자,서울특별시,서울특별시,강남구,여자,렌탈,프로모션,일반,11-19-2014,60,CMS,DES-3A,102900,계약확정,0,없음,신한은행,정상,2
4,25851817,사업자,서울특별시,서울특별시,강남구,여자,멤버십,멤버십S,렌탈재계약,10-21-2014,12,CMS,DES-1,96600,계약확정,0,없음,국민은행,정상,1


In [146]:
df_new_b.to_csv('final_business.csv')  # 최종 전처리 과정을 csv파일로 외부로 저장